
# **Clasificación de Texto con Zero-shot y Few-shot usando **Chat Completions** (OpenAI)**  


> **Objetivo.** Este cuaderno guía paso a paso cómo construir un pipeline reproducible de clasificación de texto con **LLMs** usando la ruta **simple** basada en `client.chat.completions.create(...)` y **parseo de JSON**. Se incluye control de **costos** vía límite de llamadas `MAX_CALLS`, submuestreo `N_SAMPLE`, y evaluación (accuracy, macro-F1, matriz de confusión). Se siguen principios básicos de **prompt engineering**.


In [ ]:

# ============================================================
# 0) Instalación de dependencias (si está en Colab)
#    *Descomentar si se necesitan instalar paquetes en la sesión.*
# ============================================================

!pip -q install "openai==1.*" scikit-learn pandas numpy matplotlib tqdm python-dotenv



## 1) Preparación del entorno y conexión a Google Drive (Colab)

Esta sección prepara el entorno con importaciones y, si el cuaderno corre en **Colab**, monta **Google Drive**. Así se puede leer el **mismo archivo de datos** también desde computador local.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import getpass
import os, sys, json, textwrap, time, math, random
from tqdm import tqdm
from pathlib import Path
from openai import OpenAI

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

In [ ]:
# Detección de entorno Colab y montaje de Drive
IN_COLAB = False
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive  # type: ignore
    drive.mount('/drive')
    print("✔ Drive montado en /drive")
else:
    print("ℹ No se detectó Colab; se usarán rutas locales.")


## 2) API Key de OpenAI y configuración del cliente

- Por seguridad, se recomienda **no** hardcodear credenciales.
- En Colab, se sugiere ingresar el token mediante `getpass` para la sesión.
- Utilizamos **Chat Completions** con salida JSON.

> Si la clave es de **Proyecto** (formato `sk-proj-...`), el uso se verá en el **Dashboard del Proyecto** (ícono de la llave en Colab) no en el espacio personal.


In [ ]:
# Ingreso interactivo si falta la variable de entorno.
from dotenv import load_dotenv
import os
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    if IN_COLAB:
        print("Introduzca su OPENAI_API_KEY (no se almacena en disco):")
        os.environ["OPENAI_API_KEY"] = getpass.getpass()
        # Inicializa cliente (toma OPENAI_API_KEY del entorno)
        client = OpenAI()
    else:
        api_key = os.getenv("OPENAI_API_KEY")
        # Initialize the client
        client = OpenAI(api_key=api_key)
        print("⚠ Establezca OPENAI_API_KEY en su entorno para continuar.")

if client:
    print("success")

# Modelo por defecto (ajustable según coste/capacidad)
MODEL_NAME = "gpt-4o-mini"  # Alternativas: "gpt-4o", "gpt-4.1-mini", etc.
print("Hello")


## 3) Rutas y configuración general

- Ajustar `DATA_PATH` a la **misma ruta del dataset** que usa el pipeline clásico.
- `OUTPUT_DIR` se usa para guardar artefactos (predicciones, figuras) en Drive.
- `N_SAMPLE=30` permite evaluar un subconjunto para **limitar costos**.
- `MAX_CALLS` controla el **número máximo** de llamadas a la API durante la sesión.


In [37]:
# ======= CONFIGURACIÓN DE RUTAS =======
# Ajuste DATA_PATH a la misma ubicación de su dataset en Drive (o ruta local si no usa Colab).

path = '/drive/My Drive/Colab Notebooks/infotracer_Estancia/'
if not os.path.exists(path):
    path = r"C:\Users\abdel\Downloads\concentracionIA"
    
input_file = 'data_labelled/youtube-senti-labelled.xlsx' if os.path.exists('data_labelled/youtube-senti-labelled.xlsx') else "youtube-senti-labelled-short(Sheet1).csv"

output_dir = 'results_llms'

DATA_PATH = path + input_file  if IN_COLAB else './' + input_file
OUTPUT_DIR = path + output_dir if IN_COLAB else './' + output_dir
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [38]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [65]:
# ======= CONTROL DE COSTOS =======
N_SAMPLE = 15     # número de instancias de test para pruebas rápidas
MAX_CALLS = 80    # tope de llamadas a la API (ajuste según presupuesto)
API_CALLS_USED = 0

def check_quota():
    """Verifica y consume 1 crédito de llamada a la API."""
    global API_CALLS_USED
    if API_CALLS_USED >= MAX_CALLS:
        raise RuntimeError(
            f"Se alcanzó el límite de {MAX_CALLS} llamadas a la API. "
            f"Aumente MAX_CALLS o reduzca el dataset de prueba (N_SAMPLE)."
        )
    API_CALLS_USED += 1



## 4) Lectura del dataset y EDA mínima

Asumimos que existe una columna de **texto** y una de **etiqueta**. Se implementa una heurística para detectar nombres comunes. Si las columnas con las que se está trabajando algún archivo difieren, se recomienda **ajustar manualmente** `TEXT_COL` y `LABEL_COL` tras cargar el dataset.


In [40]:

TEXT_CANDIDATES  = ["text","texto","review","content","sentence","tweet","document"]
LABEL_CANDIDATES = ["label","labels","etiqueta","etiquetas","sentiment","Sentiment","target","y"]

def load_dataset(path: str, sheet_name=0):
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"No se encontró el archivo en: {p}")

    suffix = p.suffix.lower()

    if suffix in [".csv", ".tsv"]:
        sep = "," if suffix == ".csv" else "\t"
        encodings_to_try = ["utf-8", "utf-8-sig", "cp1252", "latin-1"]
        last_err = None
        for enc in encodings_to_try:
            try:
                return pd.read_csv(p, sep=sep, encoding=enc)
            except UnicodeDecodeError as e:
                last_err = e
                continue
        # último recurso: reemplaza caracteres ilegales (pandas ≥ 2.0)
        try:
            return pd.read_csv(p, sep=sep, encoding="utf-8", encoding_errors="replace")
        except TypeError:
            # si tu pandas no acepta encoding_errors
            raise last_err

    elif suffix in [".parquet", ".pq"]:
        return pd.read_parquet(p)

    elif suffix in [".xlsx", ".xls"]:
        # .xlsx usa openpyxl; .xls requiere xlrd (<2.0) instalado
        engine = "openpyxl" if suffix == ".xlsx" else None
        return pd.read_excel(p, sheet_name=sheet_name, engine=engine)

    else:
        # Intento por defecto: CSV con detección de delimitador
        encodings_to_try = ["utf-8", "utf-8-sig", "cp1252", "latin-1"]
        last_err = None
        for enc in encodings_to_try:
            try:
                return pd.read_csv(p, sep=None, engine="python", encoding=enc)
            except UnicodeDecodeError as e:
                last_err = e
                continue
        try:
            return pd.read_csv(p, sep=None, engine="python", encoding="utf-8", encoding_errors="replace")
        except TypeError:
            raise last_err


In [41]:
df = load_dataset(DATA_PATH).copy()
print("Tamaño bruto:", len(df))

Tamaño bruto: 999


In [42]:
df = load_dataset(DATA_PATH).copy()
print("Tamaño bruto:", len(df))

Tamaño bruto: 999


In [ ]:
df

In [61]:
# Inferencia de columnas
TEXT_COL = next((c for c in TEXT_CANDIDATES if c in df.columns), None)
LABEL_COL = next((c for c in LABEL_CANDIDATES if c in df.columns), None)

if TEXT_COL is None or LABEL_COL is None:
    raise ValueError(
        f"No se detectaron columnas esperadas.\n"
        f"Candidatos texto: {TEXT_CANDIDATES}\n"
        f"Candidatos etiqueta: {LABEL_CANDIDATES}\n"
        f"Columnas disponibles: {list(df.columns)}"
    )

df = df[[TEXT_COL, LABEL_COL]].dropna().reset_index(drop=True)
display(df.head())

,text,Sentiment
0,#claudiasheinbaum __ Después de terminar el ar...,positive
1,??EN VIVO Mario Delgado. Conferencia de prensa...,neutral
2,??NO LO QUIEREN DE FISCAL! EMBESTIDA CONTRA ZA...,negative
3,ESTO LE CONTESTA A LILLY TÉLLEZ#noticias #shor...,negative
4,Claudia Sheinbaum recuerda La Linea 12 del MET...,negative


In [63]:
print("Distribución de etiquetas:")
print(df[LABEL_COL].value_counts())

Distribución de etiquetas:
Sentiment
negative    63
neutral     30
positive     7
Name: count, dtype: int64



## 5) Partición Train/Test

Se separa un conjunto de **entrenamiento** (para seleccionar ejemplos Few-shot) y un conjunto **test** (para evaluación). En **Zero-shot**, el modelo no ve ejemplos; en **Few-shot**, se inyectan ejemplos en el prompt.


In [45]:
RANDOM_SEED = 42
TEST_SIZE = 0.3

train_df, test_df = train_test_split(
    df, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=df[LABEL_COL]
)

labels = sorted(df[LABEL_COL].unique().tolist())
print("Etiquetas detectadas:", labels)
print(f"Tamaño train: {len(train_df)} | Tamaño test: {len(test_df)}")


Etiquetas detectadas: ['negative', 'neutral', 'positive']
Tamaño train: 70 | Tamaño test: 30



## 6) Submuestreo para pruebas rápidas (control de costos)

Para evitar un consumo elevado durante la etapa de prototipado, se seleccionan **30 instancias aleatorias** del conjunto de test. Este valor puede ajustarse en `N_SAMPLE`.


In [51]:
# Submuestra de test
if N_SAMPLE is not None and N_SAMPLE > 0:
    test_sample = test_df.sample(n=min(N_SAMPLE, len(test_df)), random_state=RANDOM_SEED).reset_index(drop=True)
else:
    test_sample = test_df.reset_index(drop=True)

print(f"Usando {len(test_sample)} instancias para pruebas rápidas (de {len(test_df)} totales).")
display(test_sample.head())


Usando 15 instancias para pruebas rápidas (de 30 totales).


,text,Sentiment
0,LA PATIZA DE SU VIDA??SIN MIEDO AL CONTEO DE V...,positive
1,CLAUDIA ES LA CONTINUIDAD DE LA 4T #claudiashe...,neutral
2,MÁYNEZ se BURLA de la OPOSICIÓN ???? tras RENU...,negative
3,#claudiasheinbaum #mexicanos #mexico #cuartatr...,negative
4,Segundo Debate Presidencial: Empleo e inflació...,negative



## 7) Taxonomía de etiquetas y políticas de fuera de dominio (opcional)

- Se puede trabajar un **diccionario canónico** de descripciones por etiqueta para mejorar la desambiguación. Es decir, explicar lo que significa cada etiqueta.
- Puede habilitarse una etiqueta **OUT_OF_DOMAIN** cuando el texto no encaje en ninguna clase.

Estas definiciones se incorporan al **system prompt**.


In [47]:
# Descripciones canónicas por etiqueta:
LABEL_DESCRIPTIONS = {lab: f"Definición breve y canónica de la clase '{lab}'." for lab in labels}

ALLOW_OOD = False
OOD_LABEL = "OUT_OF_DOMAIN"
if ALLOW_OOD and OOD_LABEL not in labels:
    labels_plus = labels + [OOD_LABEL]
else:
    labels_plus = labels[:]

print("Conjunto de etiquetas permitido:", labels_plus)

Conjunto de etiquetas permitido: ['negative', 'neutral', 'positive']



## 8) Prompt Engineering: **System prompt** y **User prompt**

**Principios didácticos** que guían este diseño:
- Instrucciones claras y delimitadas.
- Lista explícita de **etiquetas permitidas**.
- Formato de salida **JSON** con claves esperadas: `label`, `confidence`, `explanation`.
- **Temperature=0** para priorizar consistencia.

> En Few-shot, se agregan **k ejemplos** al *system prompt* bajo una sección de contexto. Alternativamente, también se puede hacer *few-shot* con pares de mensajes (`user`/`assistant`) para cada ejemplo; aquí se trabaja el esquema con sección de ejemplos por simplicidad.


In [53]:
def build_system_prompt(label_desc: dict, labels_allowed: list, allow_ood: bool) -> str:
    label_block = "\n".join([f"- {k}: {v}" for k, v in label_desc.items()])
    extra_ood = (
        f"\n- {OOD_LABEL}: Use esta etiqueta únicamente si el texto no encaja en ninguna clase anterior."
        if allow_ood else ""
    )
    system = f"""
Asume el rol de un experto en análisis de sentimientos de comentarios en redes sociales, centrado en el contexto político mexicano.

Eres un asistente de **clasificación de texto**. Tu tarea es asignar **una única etiqueta de sentimiento** al texto de entrada, seleccionándola de la lista permitida.
Devuelve la salida **exclusivamente** en formato JSON válido con las claves:

- "sentiment": etiqueta predicha (string; obligatoria; debe pertenecer al conjunto permitido).
- "confidence_0_5": número en [0,5] que represente tu certeza subjetiva sobre la etiqueta predicha.
- "justification": un objeto que contenga:
  - "keywords": lista de 2 a 4 palabras clave que influyen en tu decisión.
  - "spans": lista de fragmentos textuales relevantes del post.
  - "explanation": explicación concisa sobre tu justificativa para tu predicción (2-3 frases).

**Conjunto de etiquetas permitido** ({len(labels_allowed)}):
{label_block}{extra_ood}

**Reglas de decisión**
- Basándote en el **sentimiento general** del texto, escoge **una sola** etiqueta del conjunto permitido que mejor lo represente.
- Utiliza las descripciones de las etiquetas proporcionadas para guiar tu decisión.
- Si no encuentras evidencia suficiente de un sentimiento claro, el texto es ambiguo o está completamente fuera del dominio de los sentimientos definidos (solo si se habilitó '{OOD_LABEL}'), utiliza '{OOD_LABEL}'.
**Formato de salida obligatorio**
Devuelve **solo** JSON, sin prosa adicional, p. ej.:

{{
    "sentiment": "negative",
  "confidence_0_5": 4,
  "justification": {{
    "keywords": ["indignante", "corrupción"],
    "spans": ["¡Qué indignante!", "habla de corrupción"],
    "explanation": "El texto expresa claramente indignación y menciona corrupción, lo que indica un sentimiento negativo."
  }}
}}
""".strip()
    return system

def build_user_prompt(texto: str) -> str:
    user = f"""
Clasifica el siguiente documento y responde **exclusivamente** con el JSON solicitado:

```texto
{texto}
```
""".strip()
    return user

SYSTEM_PROMPT = build_system_prompt(LABEL_DESCRIPTIONS, labels_plus, ALLOW_OOD)


## 9) Funciones auxiliares: parseo robusto de JSON y cuota de llamadas

- `parse_json_strict`: intenta decodificar el JSON; hace limpiezas mínimas si hay backticks u otros delimitadores.
- `check_quota`: asegura que no se exceda `MAX_CALLS`.


In [49]:
def parse_json_strict(raw: str) -> dict:
    """
    Intenta parsear JSON desde un string.
    - Elimina backticks y espacios innecesarios.
    - Lanza ValueError si no logra decodificar.
    """
    if raw is None:
        raise ValueError("Respuesta vacía.")
    cleaned = raw.strip()
    # Limpiezas frecuentes (bloques con ```json ... ```)
    if cleaned.startswith("```"):
        cleaned = cleaned.strip("`")
        # Si queda un prefijo 'json\n', eliminarlo
        if cleaned.lower().startswith("json"):
            cleaned = cleaned[4:].lstrip()
    # Intento de parseo directo
    return json.loads(cleaned)



## 10) Llamadas a la API (Chat Completions) — Zero-shot y Few-shot

- Se usa `response_format={"type":"json_object"}` para **favorecer** que la salida sea JSON válido.
- En **Zero-shot**, el *system prompt* es el base.
- En **Few-shot**, se **inyecta** una sección de ejemplos al final del *system prompt*.


In [55]:
def llm_classify_one_zero(texto: str,
                          temperature: float = 0.0,
                          model: str = MODEL_NAME,
                          max_retries: int = 3,
                          sleep_base: float = 1.5):
    """
    Clasifica un texto sin ejemplos (Zero-shot) usando Chat Completions.
    Devuelve un dict con {label, confidence_0_5, explanation, raw}.
    """
    user_prompt = build_user_prompt(texto)
    last_err = None
    for attempt in range(max_retries):
        try:
            check_quota()
            resp = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=temperature,
                response_format={"type": "json_object"}  # favorece JSON válido
            )
            raw = resp.choices[0].message.content
            data = parse_json_strict(raw)

            label = data.get("label")
            if label not in labels_plus:
                label = OOD_LABEL if ALLOW_OOD else random.choice(labels_plus)
            return {
                "label": label,
                "confidence_0_5": data.get("confidence_0_5"),
                "justification": data.get("justification", {
                    "keywords": [],
                    "spans": [],
                    "explanation": data.get("explanation", "")
                }),
                "raw": data
            }
        except Exception as e:
            last_err = e
            print(f"[llm_classify_one_zero] intento {attempt+1} falló: {e}")
            time.sleep(sleep_base * (2 ** attempt))

    # Falla definitiva
    return {
        "label": OOD_LABEL if ALLOW_OOD else random.choice(labels_plus),
        "confidence_0_5": None,
        "justification": {
            "keywords": f"fallback: {last_err}",
            "spans": f"fallback: {last_err}",
            "explanation": f"fallback: {last_err}"
        },
        "raw": None,
    }


def sample_few_shot_examples(train: pd.DataFrame, label_col: str, text_col: str,
                             k: int, seed: int=42):
    """
    Selecciona k ejemplos por clase desde train para Few-shot.
    Estrategia: muestreo estratificado simple.
    """
    examples = []
    for lab in labels:
        subset = train[train[label_col]==lab]
        kk = min(k, len(subset))
        if kk == 0:
            continue
        exs = subset.sample(n=kk, random_state=seed)
        for _, row in exs.iterrows():
            examples.append({"label": lab, "text": str(row[text_col])})
    return examples


def build_few_shot_system_prompt(base_system: str, examples: list) -> str:
    """
    Agrega una sección con ejemplos al final del system prompt.
    """
    parts = [base_system, "\n=== EJEMPLOS DE CONTEXTO (Few-shot) ==="]
    for i, ex in enumerate(examples, 1):
        parts.append(
            f"\nEjemplo {i}:\n"
            f"Texto:\n\"\"\"\n{ex['text']}\n\"\"\"\n"
            f"Etiqueta esperada: {ex['label']}"
        )
    parts.append("\n=== FIN DE EJEMPLOS ===")
    return "\n".join(parts)


def llm_classify_one_fewshot(texto: str, examples: list,
                             temperature: float = 0.0, model: str = MODEL_NAME,
                             max_retries: int = 3, sleep_base: float = 1.5):
    """
    Clasifica un texto **con** ejemplos (Few-shot) inyectados en el system prompt.
    Devuelve una etiqueta.
    """
    user_prompt = build_user_prompt(texto)
    sys_prompt_fs = build_few_shot_system_prompt(SYSTEM_PROMPT, examples)

    last_err = None
    for attempt in range(max_retries):
        try:
            check_quota()
            resp = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": sys_prompt_fs},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=temperature,
                response_format={"type": "json_object"}
            )
            raw = resp.choices[0].message.content
            data = parse_json_strict(raw)
            label = data.get("label")
            if label not in labels_plus:
                label = OOD_LABEL if ALLOW_OOD else random.choice(labels_plus)
            return label
        except Exception as e:
            last_err = e
            print(f"[llm_classify_one_fewshot] intento {attempt+1} falló: {e}")
            time.sleep(sleep_base * (2 ** attempt))

    return OOD_LABEL if ALLOW_OOD else random.choice(labels_plus)



## 11) Inferencia por lotes y métricas de evaluación

Se implementan funciones para ejecutar Zero-shot y Few-shot sobre el subconjunto de prueba y calcular métricas clásicas.


In [ ]:
def run_zero_shot(df_in: pd.DataFrame, text_col: str, label_col: str) -> pd.DataFrame:
    preds = []
    for txt in tqdm(df_in[text_col].tolist(), desc="Zero-shot inferencia"):
        out = llm_classify_one_zero(txt, temperature=0.0, model=MODEL_NAME)
        preds.append(out["label"])
    out_df = df_in.copy()
    out_df["pred_zero_shot"] = preds
    return out_df


def run_few_shot(train_df: pd.DataFrame, test_df: pd.DataFrame,
                 text_col: str, label_col: str, k_shots: int = 3) -> pd.DataFrame:
    examples = sample_few_shot_examples(train_df, label_col, text_col, k=k_shots, seed=RANDOM_SEED)
    preds = []
    for txt in tqdm(test_df[text_col].tolist(), desc=f"Few-shot (k={k_shots}) inferencia"):
        preds.append(llm_classify_one_fewshot(txt, examples, temperature=0.0, model=MODEL_NAME))
    out_df = test_df.copy()
    out_df[f"pred_few_shot_k{k_shots}"] = preds
    return out_df


def plot_confusion(cm: np.ndarray, labels_axis: list, title: str):
    fig = plt.figure(figsize=(6,5))
    plt.imshow(cm, interpolation='nearest')
    plt.title(title)
    plt.xlabel("Predicción"); plt.ylabel("Verdadero")
    plt.xticks(range(len(labels_axis)), labels_axis, rotation=45, ha="right")
    plt.yticks(range(len(labels_axis)), labels_axis)
    for (i, j), z in np.ndenumerate(cm):
        plt.text(j, i, str(z), ha='center', va='center')
    plt.tight_layout()
    plt.show()

### Zero-shot

In [66]:
# ==== Zero-shot ====
zero_df = run_zero_shot(test_sample, TEXT_COL, LABEL_COL)
acc_z  = accuracy_score(zero_df[LABEL_COL], zero_df["pred_zero_shot"])
f1m_z  = f1_score(zero_df[LABEL_COL], zero_df["pred_zero_shot"], average="macro")
print(f"[Zero-shot] accuracy={acc_z:.4f} | macro-F1={f1m_z:.4f}")
print(classification_report(zero_df[LABEL_COL], zero_df["pred_zero_shot"]))

Zero-shot inferencia:   0%|                                                                                                                | 0/15 [00:00<?, ?it/s]

[llm_classify_one_zero] intento 1 falló: name 'client' is not defined
[llm_classify_one_zero] intento 2 falló: name 'client' is not defined
[llm_classify_one_zero] intento 3 falló: name 'client' is not defined


Zero-shot inferencia:   0%|                                                                                                                | 0/15 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
cm_z = confusion_matrix(zero_df[LABEL_COL], zero_df["pred_zero_shot"], labels=labels_plus)
plot_confusion(cm_z, labels_plus, "Matriz de confusión — Zero-shot")

### Few-shot

In [ ]:
# ==== Few-shot ====
K_SHOTS = 3
few_df = run_few_shot(train_df, test_sample, TEXT_COL, LABEL_COL, k_shots=K_SHOTS)
acc_f  = accuracy_score(few_df[LABEL_COL], few_df[f"pred_few_shot_k{K_SHOTS}"])
f1m_f  = f1_score(few_df[LABEL_COL], few_df[f"pred_few_shot_k{K_SHOTS}"], average="macro")
print(f"[Few-shot k={K_SHOTS}] accuracy={acc_f:.4f} | macro-F1={f1m_f:.4f}")
print(classification_report(few_df[LABEL_COL], few_df[f"pred_few_shot_k{K_SHOTS}"]))

In [ ]:
cm_f = confusion_matrix(few_df[LABEL_COL], few_df[f"pred_few_shot_k{K_SHOTS}"], labels=labels_plus)
plot_confusion(cm_f, labels_plus, f"Matriz de confusión — Few-shot (k={K_SHOTS})")


## 12) Comparativa y análisis de desacuerdos

Se contrasta Zero-shot vs Few-shot y se listan algunos casos donde discrepan para facilitar el **análisis cualitativo**.


In [60]:
compare_df = test_sample.copy().reset_index(drop=True)
compare_df["pred_zero_shot"] = zero_df["pred_zero_shot"].values
compare_df[f"pred_few_shot_k{K_SHOTS}"] = few_df[f"pred_few_shot_k{K_SHOTS}"].values
compare_df

NameError: name 'few_df' is not defined

In [ ]:
def disagreement_sample(df_in: pd.DataFrame, n: int = 10) -> pd.DataFrame:
    mask = df_in["pred_zero_shot"] != df_in[f"pred_few_shot_k{K_SHOTS}"]
    if mask.sum() == 0:
        return pd.DataFrame(columns=[TEXT_COL, LABEL_COL, "pred_zero_shot", f"pred_few_shot_k{K_SHOTS}"])
    return df_in[mask].sample(n=min(n, mask.sum()), random_state=RANDOM_SEED)[
        [TEXT_COL, LABEL_COL, "pred_zero_shot", f"pred_few_shot_k{K_SHOTS}"]
    ]

summary = pd.DataFrame({
    "setting": ["Zero-shot", f"Few-shot (k={K_SHOTS})"],
    "accuracy": [acc_z, acc_f],
    "macro_f1": [f1m_z, f1m_f],
})

display(summary)

In [ ]:
display(disagreement_sample(compare_df, n=8))


## 13) Guardado de artefactos (CSV + figuras)

Los resultados se guardan en `OUTPUT_DIR`. Si se trabaja en Colab con Google Drive montado, los archivos quedan disponibles en la carpeta de Drive.


In [ ]:
zero_csv = os.path.join(OUTPUT_DIR, "predicciones_zero_shot.csv")
few_csv  = os.path.join(OUTPUT_DIR, f"predicciones_few_shot_k{K_SHOTS}.csv")

zero_df.to_csv(zero_csv, index=False)
few_df.to_csv(few_csv, index=False)

print("Guardados:")
print(" -", zero_csv)
print(" -", few_csv)



## 14) Notas operativas y buenas prácticas

- **Determinismo relativo**: con `temperature=0` se favorece consistencia, pero pequeñas variaciones pueden ocurrir.
- **Costo y latencia**: `N_SAMPLE` y `MAX_CALLS` permiten limitar el gasto en prototipos; ampliar cuando se necesite evaluación completa.
- **Trazabilidad**: usar claves de **Proyecto** (`sk-proj-...`) para agrupar en un Dashboard específico y facilitar monitoreo.
- **Seguridad**: manejar el token vía variables de entorno; evitar guardarlo en texto plano.
- **Transferencia**: mantener un vocabulario canónico de etiquetas y descripciones (`LABEL_DESCRIPTIONS`) para facilitar reutilización entre dominios.


# Contrafactual

In [ ]:
def llm_evaluate_counterfactual(original_text: str, flipped_text: str, transformation_type: str, explanation: str,
                                temperature: float = 0.0, model: str = MODEL_NAME,
                                max_retries: int = 3, sleep_base: float = 1.5):
    """
    Evalúa una versión contrafactual de un texto usando Chat Completions.
    Devuelve un dict con las evaluaciones y el raw JSON.
    """
    system_prompt_cf = """
    Estás evaluando un texto contrafactual. El texto original y su contrafactual son proporcionados. El
Estás evaluando una versión sintética de un texto. El mensaje sintético es una versión del original con el sentimiento invertido.
Evalúa la calidad del mensaje sintético según cuatro criterios utilizando un rango de [0,1]:
1. Fluidad - ¿El mensaje sintético es graticalmente correcto y puede leerse con facilidad?
2. Naturalidad - ¿Suena creíble que lo haya escrito un humano?
3. Claridad del cambio de sentimiento - ¿El sentimiento ha cambiado con respecto al original?
4. Conservación del significado - ¿Se conserva el significado principal aparte del sentimiento?
Devuelva SOLO este JSON:
{
"fluidez": 0 o 1,
"naturalidad": 0 o 1,
"claridad_cambio_sentimiento": 0 o 1,
"conservación_significado": 0 o 1,
"comentario_annotador": "comentario opcional (string)"
}
""".strip()

    user_prompt_cf = f"""
Mensaje Original: "{original_text}"
Mensaje Sintético: "{flipped_text}"
Transformation Type: {transformation_type}
GPT-4 Explanation for the Flip: "{explanation}"
""".strip()

    last_err = None
    for attempt in range(max_retries):
        try:
            check_quota()
            resp = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_prompt_cf},
                    {"role": "user", "content": user_prompt_cf},
                ],
                temperature=temperature,
                response_format={"type": "json_object"}
            )
            raw = resp.choices[0].message.content
            data = parse_json_strict(raw)
            return {
                "fluidez": data.get("fluidez"),
                "naturalidad": data.get("naturalidad"),
                "claridad_cambio_sentimiento": data.get("claridad_cambio_sentimiento"),
                "conservación_significado": data.get("conservación_significado"),
                "comentario_annotador": data.get("comentario_annotador", ""),
                "raw": data
            }
        except Exception as e:
            last_err = e
            print(f"[llm_evaluate_counterfactual] intento {attempt+1} falló: {e}")
            time.sleep(sleep_base * (2 ** attempt))

    return {
        "fluidez": None,
        "naturalidad": None,
        "claridad_cambio_sentimiento": None,
        "conservación_significado": None,
        "comentario_annotador": f"fallback: {last_err}",
        "raw": None,
    }

In [ ]:
# Aplicar la evaluación contrafactual a cada fila de compare_df

evaluation_results_zero_shot = []
evaluation_results_few_shot = []

# Iterar sobre las filas del DataFrame
for index, row in tqdm(compare_df.iterrows(), total=len(compare_df), desc="Evaluando contrafactuales"):
    original_text = row[TEXT_COL]
    # Evaluar Zero-shot prediction as the flipped text
    flipped_text_zero = row["pred_zero_shot"]
    result_zero = llm_evaluate_counterfactual(
        original_text=original_text,
        flipped_text=flipped_text_zero,
        transformation_type="Zero-shot Prediction", # Puedes ajustar esto si tienes un tipo de transformación más específico
        explanation="" # La explicación puede ser vacía o basada en la justificación del modelo si la guardaste
    )
    evaluation_results_zero_shot.append(result_zero)

    # Evaluar Few-shot prediction as the flipped text
    flipped_text_few = row[f"pred_few_shot_k{K_SHOTS}"]
    result_few = llm_evaluate_counterfactual(
        original_text=original_text,
        flipped_text=flipped_text_few,
        transformation_type=f"Few-shot Prediction (k={K_SHOTS})", # Puedes ajustar esto
        explanation="" # La explicación puede ser vacía o basada en la justificación del modelo
    )
    evaluation_results_few_shot.append(result_few)


# Convertir los resultados a DataFrames y unirlos al original
eval_df_zero = pd.DataFrame(evaluation_results_zero_shot)
eval_df_zero.columns = [f"eval_zero_shot_{col}" for col in eval_df_zero.columns]

eval_df_few = pd.DataFrame(evaluation_results_few_shot)
eval_df_few.columns = [f"eval_few_shot_{col}" for col in eval_df_few.columns]


# Unir los resultados al DataFrame original (compare_df)
# Usamos .copy() para evitar SettingWithCopyWarning
compare_df_evaluated = compare_df.copy()
compare_df_evaluated = pd.concat([compare_df_evaluated, eval_df_zero, eval_df_few], axis=1)

# Mostrar el DataFrame con los resultados de la evaluación
display(compare_df_evaluated.head())

In [ ]:
# Ejemplo de uso de la función llm_evaluate_counterfactual

# Datos de ejemplo (reemplaza con tus datos reales)
original_text_example = "Me encantó la película, fue muy emocionante."
flipped_text_example = "Odié la película, fue muy aburrida."
transformation_type_example = "Manual Sentiment Flip"
explanation_example = "Se reemplazaron palabras positivas por antónimos negativos."

# Llama a la función de evaluación
evaluation_result = llm_evaluate_counterfactual(
    original_text=original_text_example,
    flipped_text=flipped_text_example,
    transformation_type=transformation_type_example,
    explanation=explanation_example
)

# Muestra el resultado de la evaluación
print("Resultado de la evaluación contrafactual:")
display(evaluation_result)

esto se lo pedi a gemini jeje pero así da pauta de moverle ára el generation y el filtering

In [ ]:
def llm_generate_counterfactual(original_text: str, original_sentiment: str,
                                temperature: float = 0.0, model: str = MODEL_NAME,
                                max_retries: int = 3, sleep_base: float = 1.5):
    """
    Generates counterfactual versions of a text with flipped sentiment using Chat Completions.
    Returns a list of dicts with the counterfactual texts and explanations, and the raw JSON.
    """
    system_prompt_cf_gen = """
    You are an NLP assistant helping researchers generate high-quality counterfactual examples for sentiment classification. Given a text and its sentiment (positive or negative), generate 3 distinct versions that flip the sentiment. Only modify necessary components. Preserve fluency and realism. Respect informal tone. You may flip sentiment by changing components such as: - keywords, phrases, negation, intent framing, tone (e.g., sarcasm), sentiment valence, emojis/icons, code-mixing.
    Devuelva SOLO este JSON:
    [
      {{
        "cf_text": "...",
        "components_changed": ["...", "..."],
        "flip_explanation": "..."
      }},
      {{
        "cf_text": "...",
        "components_changed": ["...", "..."],
        "flip_explanation": "..."
      }},
      {{
        "cf_text": "...",
        "components_changed": ["...", "..."],
        "flip_explanation": "..."
      }}
    ]
    """.strip()

    user_prompt_cf_gen = f"""
    Input:
    Original message: "{original_text}"
    Original sentiment:"{original_sentiment}"
    """.strip()

    last_err = None
    for attempt in range(max_retries):
        try:
            check_quota()
            resp = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_prompt_cf_gen},
                    {"role": "user", "content": user_prompt_cf_gen},
                ],
                temperature=temperature,
                response_format={"type": "json_object"}
            )
            raw = resp.choices[0].message.content
            data = parse_json_strict(raw) # Assuming parse_json_strict can handle a list of dicts
            return {
                "counterfactuals": data,
                "raw": data
            }
        except Exception as e:
            last_err = e
            print(f"[llm_generate_counterfactual] intento {attempt+1} falló: {e}")
            time.sleep(sleep_base * (2 ** attempt))

    return {
        "counterfactuals": [],
        "raw": None,
        "error": f"fallback: {last_err}"
    }

# Example Usage (you can uncomment and run this to test the function)
# original_text_gen_example = "Me encantó la película, fue muy emocionante."
# original_sentiment_gen_example = "positive"

# generated_counterfactuals = llm_generate_counterfactual(
#     original_text=original_text_gen_example,
#     original_sentiment=original_sentiment_gen_example
# )

# print("\nGenerated Counterfactuals:")
# display(generated_counterfactuals)

In [ ]:
def llm_filter_counterfactual(original_text: str, original_sentiment: str,
                              cf_candidates: list[str],
                              temperature: float = 0.0, model: str = MODEL_NAME,
                              max_retries: int = 3, sleep_base: float = 1.5):
    """
    Filters the best counterfactual from a list of candidates using Chat Completions.
    Returns a dict with the selected counterfactual, justification, and predicted sentiment.
    """
    system_prompt_cf_filter = """
    You are a sentiment evaluation assistant. Your task is to select the best counterfactual rewrite of a message.
    RESPONSE FORMAT (JSON only):
    {{
    "selected_cf": "...",
    "justification": "...",
    "predicted_sentiment": "Positive / Negative"
    }}
    """.strip()

    cf_list_str = "\n".join([f"{i+1}. \"{cf}\"" for i, cf in enumerate(cf_candidates)])

    user_prompt_cf_filter = f"""
    ORIGINAL MESSAGE
    "{original_text}"
    (Sentiment: {original_sentiment})

    COUNTERFACTUAL CANDIDATES
    {cf_list_str}

    INSTRUCTIONS
    Your goal is to identify which counterfactual most effectively flips the sentiment while remaining realistic and fluent.
    - Flip sentiment plausibly
    - Sound natural in social media comments
    - Preserve meaning/context where possible
    """.strip()

    last_err = None
    for attempt in range(max_retries):
        try:
            check_quota()
            resp = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_prompt_cf_filter},
                    {"role": "user", "content": user_prompt_cf_filter},
                ],
                temperature=temperature,
                response_format={"type": "json_object"}
            )
            raw = resp.choices[0].message.content
            data = parse_json_strict(raw)
            return {
                "selected_cf": data.get("selected_cf"),
                "justification": data.get("justification"),
                "predicted_sentiment": data.get("predicted_sentiment"),
                "raw": data
            }
        except Exception as e:
            last_err = e
            print(f"[llm_filter_counterfactual] intento {attempt+1} falló: {e}")
            time.sleep(sleep_base * (2 ** attempt))

    return {
        "selected_cf": None,
        "justification": f"fallback: {last_err}",
        "predicted_sentiment": None,
        "raw": None,
    }

# Example Usage (you can uncomment and run this to test the function)
# original_text_filter_example = "Me encantó la película, fue muy emocionante."
# original_sentiment_filter_example = "positive"
# cf_candidates_example = [
#     "Odié la película, fue muy aburrida.",
#     "La película no estuvo tan buena como esperaba.",
#     "Fue una pérdida de tiempo ver esa película."
# ]

# filtered_counterfactual = llm_filter_counterfactual(
#     original_text=original_text_filter_example,
#     original_sentiment=original_sentiment_filter_example,
#     cf_candidates=cf_candidates_example
# )

# print("\nFiltered Counterfactual:")
# display(filtered_counterfactual)